In [57]:
import numpy as np
import pandas as pd
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import GridSearchCV,RandomizedSearchCV
from sklearn.neighbors import KNeighborsRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.svm import SVR
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import Lasso
from sklearn.linear_model import Ridge
import warnings
warnings.filterwarnings("ignore")

In [58]:
df_train = pd.read_excel("Restaurant_Data_Train.xlsx")

In [59]:
df_train.head()

,TITLE,RESTAURANT_ID,CUISINES,TIME,CITY,LOCALITY,RATING,VOTES,COST
0,CASUAL DINING,9438,"Malwani, Goan, North Indian","11am – 4pm, 7:30pm – 11:30pm (Mon-Sun)",Thane,Dombivali East,3.6,49 votes,1200
1,"CASUAL DINING,BAR",13198,"Asian, Modern Indian, Japanese",6pm – 11pm (Mon-Sun),Chennai,Ramapuram,4.2,30 votes,1500
2,CASUAL DINING,10915,"North Indian, Chinese, Biryani, Hyderabadi","11am – 3:30pm, 7pm – 11pm (Mon-Sun)",Chennai,Saligramam,3.8,221 votes,800
3,QUICK BITES,6346,"Tibetan, Chinese",11:30am – 1am (Mon-Sun),Mumbai,Bandra West,4.1,24 votes,800
4,DESSERT PARLOR,15387,Desserts,11am – 1am (Mon-Sun),Mumbai,Lower Parel,3.8,165 votes,300


In [60]:
df_test = pd.read_excel("Restaurant_Data_Test.xlsx")

df_test.head()

,TITLE,RESTAURANT_ID,CUISINES,TIME,CITY,LOCALITY,RATING,VOTES
0,CASUAL DINING,4085,"North Indian, Chinese, Mughlai, Kebab",12noon – 12midnight (Mon-Sun),Noida,Sector 18,4.3,564 votes
1,QUICK BITES,12680,"South Indian, Fast Food, Pizza, North Indian",7am – 12:30AM (Mon-Sun),Mumbai,Grant Road,4.2,61 votes
2,CASUAL DINING,1411,"North Indian, Seafood, Biryani, Chinese",11am – 11:30pm (Mon-Sun),Mumbai,Marine Lines,3.8,350 votes
3,None,204,Biryani,"9am – 10pm (Mon, Wed, Thu, Fri, Sat, Sun), 10:...",Faridabad,NIT,3.8,1445 votes
4,QUICK BITES,13453,"South Indian, Kerala",11am – 10pm (Mon-Sun),Kochi,Kaloor,3.6,23 votes


In [61]:
df_train.shape

(12690, 9)

In [62]:
df_train["TIME"].unique()

array(['11am – 4pm, 7:30pm – 11:30pm (Mon-Sun)', '6pm – 11pm (Mon-Sun)',
       '11am – 3:30pm, 7pm – 11pm (Mon-Sun)', ...,
       '12:1pm – 4am (Mon-Sun)', '12:30pm – 9:45pm (Mon-Sun)',
       '12noon – 1am (Mon-Fri),11am – 5pm, 7pm – 1am...'], dtype=object)

In [63]:
df_train["TIME"].value_counts()

11am – 11pm (Mon-Sun)                                 1415
12noon – 12midnight (Mon-Sun)                          469
12noon – 11pm (Mon-Sun)                                407
11am – 12midnight (Mon-Sun)                            310
10am – 10pm (Mon-Sun)                                  221
                                                      ... 
7am – 11pm (Mon-Sat),7am – 3:30pm (Sun)                  1
6am – 7pm (Mon-Sun)                                      1
Closed (Mon),10am – 11:30pm (Tue, Wed, Fri, Sat...       1
5pm – 11:45pm (Mon),5pm – 11:45pm, 12midnight...         1
12:30pm – 11:55pm (Mon-Sun)                              1
Name: TIME, Length: 2689, dtype: int64

In [64]:
df_train.TIME.values

array(['11am – 4pm, 7:30pm – 11:30pm (Mon-Sun)', '6pm – 11pm (Mon-Sun)',
       '11am – 3:30pm, 7pm – 11pm (Mon-Sun)', ...,
       '12noon – 12:30AM (Mon-Sun)', '6am – 10:45pm (Mon-Sun)',
       '12:30pm – 11pm (Mon-Sun)'], dtype=object)

In [65]:
df_train.tail()

,TITLE,RESTAURANT_ID,CUISINES,TIME,CITY,LOCALITY,RATING,VOTES,COST
12685,QUICK BITES,13228,"North Indian, Burger, Kebab","12noon – 12midnight (Mon, Tue, Wed, Thu, Sun)...",Hyderabad,Gachibowli,3.8,546 votes,500
12686,"CASUAL DINING,BAR",9686,"Goan, Continental","12noon – 1am (Mon-Fri),11am – 5pm, 7pm – 1am...",Mumbai,Bandra Kurla Complex,4.3,1214 votes,1800
12687,LOUNGE,11133,"Finger Food, Continental, Asian, Chinese",12noon – 12:30AM (Mon-Sun),Navi Mumbai,Vashi,4.0,608 votes,1300
12688,CASUAL DINING,6134,"North Indian, South Indian, Chinese, Street Food",6am – 10:45pm (Mon-Sun),Chennai,Maduravoyal,3.5,32 votes,400
12689,CASUAL DINING,6430,"North Indian, Chinese",12:30pm – 11pm (Mon-Sun),Hyderabad,Vanasthalipuram,3.4,677 votes,600


In [66]:
df_train["RATING"].values

array(['3.6', '4.2', '3.8', ..., '4.0', '3.5', '3.4'], dtype=object)

In [67]:
df_train["RATING"].unique()

array(['3.6', '4.2', '3.8', '4.1', '4.0', '4.3', '3.9', '3.3', '3.4', '-',
       '4.5', '3.5', '4.4', '2.7', '3.7', '4.7', 'NEW', '3.1', '2.5',
       '4.6', '2.8', nan, '3.0', '3.2', '2.6', '2.9', '4.9', '4.8', '2.4',
       '2.3', '2.0', '2.1', '2.2'], dtype=object)

In [68]:
df_train["CITY"].unique()

array(['Thane', 'Chennai', 'Mumbai', 'Bangalore', 'Gurgaon', 'Hyderabad',
       'Kochi', 'Thane West', 'Andheri Lokhandwala', 'New Delhi',
       'Andheri West', 'Malad East', '682036', 'Bangalor', 'Navi Mumbai',
       'Bandra West', 'Delhi', 'Noida', 'Bangalore-560066',
       'Secunderabad', nan, 'India', 'Madhuranagar', 'Chennai Teynampet',
       'Faridabad', 'Chembur.', 'Maharashtra', 'opp gurudwara Shakurpur',
       'Telagana Land Line:040-48507016', 'Ghaziabad', 'Karnataka',
       'Kerala', 'Edappally', 'Kadavanthra', 'Ernakulam Circle kochi',
       'Bengalore', 'Near Reliance Fresh', 'Kilpauk', 'Bengaluru',
       'Kothaguda', 'Goregaon West', 'Banglore', 'Tamil Nadu', 'Kakkanad',
       'Kochi Elamkulam', 'Outer Ring Road', 'Mulund East',
       'Secunderabad main road near signal NMREC COLLEGE', 'Telangana',
       'Ponnuruni Kochi', 'Gachibowli', 'Semmancheri',
       '5th Main Teachers Colony Koramangala Block 1 Bangalore 560034',
       'Mumbai Mahim', 'Powai (Next to

# Exploratory Data Analysis

1. Data Analysis : 
    
   * We will combine the training and test data are combined for further analysis.
   * For the features TITLE and CUISINES we will identify the maximum number of items listed in a 
      single cell and then split the features in to as many new features.
   * The NANs in the CITY and LOCALITY columns are replaced by "NOT AVAILABLE".
   * Also we will identify  the unique values in TITLE, CUISINES, CITY and LOALITY 
     which are to be used in encoding in the Data Preprocesing part.


    

In [69]:
#  checking the features in training Datasets


print("\nEDA on Train Set\n")
print("#############################################################################################")
print("\nFeatures/Columns  :  \n", df_train.columns)
print("\n\nNumber of Features\Columns  : ", len(df_train.columns))
print("\nNumber of Rows : ", len(df_train))
print("\n\nData types : \n", df_train.dtypes)
print("\n Contains Nan/Empty cells :", df_train.isnull().values.any())
print("\nTotal empty cells by column :\n", df_train.isnull().sum(), "\n\n")


EDA on Train Set

#############################################################################################

Features/Columns  :  
 Index(['TITLE', 'RESTAURANT_ID', 'CUISINES', 'TIME', 'CITY', 'LOCALITY',
       'RATING', 'VOTES', 'COST'],
      dtype='object')


Number of Features\Columns  :  9

Number of Rows :  12690


Data types : 
 TITLE            object
RESTAURANT_ID     int64
CUISINES         object
TIME             object
CITY             object
LOCALITY         object
RATING           object
VOTES            object
COST              int64
dtype: object

 Contains Nan/Empty cells : True

Total empty cells by column :
 TITLE               0
RESTAURANT_ID       0
CUISINES            0
TIME                0
CITY              112
LOCALITY           98
RATING              2
VOTES            1204
COST                0
dtype: int64 




In [70]:
#  checking the features in test Datasets


print("\nEDA on Test Set\n")
print("#############################################################################################")
print("\nFeatures/Columns  :  \n", df_test.columns)
print("\n\nNumber of Features\Columns  : ", len(df_test.columns))
print("\nNumber of Rows : ", len(df_test))
print("\n\nData types : \n", df_test.dtypes)
print("\n Contains Nan/Empty cells :", df_test.isnull().values.any())
print("\nTotal empty cells by column :\n", df_test.isnull().sum(), "\n\n")


EDA on Test Set

#############################################################################################

Features/Columns  :  
 Index(['TITLE', 'RESTAURANT_ID', 'CUISINES', 'TIME', 'CITY', 'LOCALITY',
       'RATING', 'VOTES'],
      dtype='object')


Number of Features\Columns  :  8

Number of Rows :  4231


Data types : 
 TITLE            object
RESTAURANT_ID     int64
CUISINES         object
TIME             object
CITY             object
LOCALITY         object
RATING           object
VOTES            object
dtype: object

 Contains Nan/Empty cells : True

Total empty cells by column :
 TITLE              0
RESTAURANT_ID      0
CUISINES           0
TIME               0
CITY              35
LOCALITY          30
RATING             2
VOTES            402
dtype: int64 




# Data Analysing



In [71]:
# Combining training set and test sets for analysing data and finding patterns

data_temp = [df_train[["TITLE", "RESTAURANT_ID","CUISINES","TIME","CITY","LOCALITY","RATING","VOTES"]],df_test]

In [72]:
data_temp = pd.concat(data_temp)

In [73]:
data_temp

,TITLE,RESTAURANT_ID,CUISINES,TIME,CITY,LOCALITY,RATING,VOTES
0,CASUAL DINING,9438,"Malwani, Goan, North Indian","11am – 4pm, 7:30pm – 11:30pm (Mon-Sun)",Thane,Dombivali East,3.6,49 votes
1,"CASUAL DINING,BAR",13198,"Asian, Modern Indian, Japanese",6pm – 11pm (Mon-Sun),Chennai,Ramapuram,4.2,30 votes
2,CASUAL DINING,10915,"North Indian, Chinese, Biryani, Hyderabadi","11am – 3:30pm, 7pm – 11pm (Mon-Sun)",Chennai,Saligramam,3.8,221 votes
3,QUICK BITES,6346,"Tibetan, Chinese",11:30am – 1am (Mon-Sun),Mumbai,Bandra West,4.1,24 votes
4,DESSERT PARLOR,15387,Desserts,11am – 1am (Mon-Sun),Mumbai,Lower Parel,3.8,165 votes
...,...,...,...,...,...,...,...,...
4226,CASUAL DINING,9057,"North Indian, Mughlai, Chinese",11:30am – 11:30pm (Mon-Sun),New Delhi,Punjabi Bagh,3.9,287 votes
4227,None,1247,"Biryani, North Indian, Sandwich, Salad, Wraps",11am – 1am (Mon-Sun),Bangalore,HSR Layout,4.3,469 votes
4228,QUICK BITES,8617,"Continental, North Indian",9:30am – 10:30pm (Mon-Sun),Faridabad,Sector 86,3.7,53 votes
4229,QUICK BITES,6485,"Rolls, Beverages","11am – 11:30pm (Mon, Tue, Wed, Thu, Sat, Sun),...",Kochi,Kochi,-,NaN


In [74]:
# Analysing Titles

titles = list(data_temp['TITLE'])

# Finding Maximum number of titles mentioned in a single cell

maxim = 1
for i in titles:
    if len(i.split(',')) > maxim:
        maxim = len(i.split(','))

print("\n\nMaximum Titles in a Cell : ", maxim) 

all_titles = []

for i in titles :
    if len(i.split(',')) == 1:
        all_titles.append(i.split(',')[0].strip().upper())
    else:
        for x in range(len(i.split(','))):
            all_titles.append(i.split(',')[x].strip().upper())
            
print("\n\nNumber of Unique Titles : ", len(pd.Series(all_titles).unique()))
print("\n\nUnique Titles:\n", pd.Series(all_titles).unique())

all_titles = list(pd.Series(all_titles).unique())



Maximum Titles in a Cell :  2


Number of Unique Titles :  25


Unique Titles:
 ['CASUAL DINING' 'BAR' 'QUICK BITES' 'DESSERT PARLOR' 'CAFÉ'
 'MICROBREWERY' 'BEVERAGE SHOP' 'IRANI CAFE' 'BAKERY' 'NONE' 'PUB'
 'FINE DINING' 'SWEET SHOP' 'LOUNGE' 'FOOD COURT' 'FOOD TRUCK' 'MESS'
 'KIOSK' 'CLUB' 'CONFECTIONERY' 'DHABA' 'MEAT SHOP' 'COCKTAIL BAR'
 'PAAN SHOP' 'BHOJANALYA']


In [75]:
# Analysing cuisines

cuisines = list(data_temp["CUISINES"])

maxim = 1

for i in cuisines:
    if len(i.split(',')) > maxim:
        maxim = len(i.split(','))
        
print("\n\nMaximum cuisines in a cell : ",maxim)        


all_cuisines = []

for i in cuisines :
    if len(i.split(',')) == 1:
        all_cuisines.append(i.split(',')[0].strip().upper())
    else:
        for x in range(len(i.split(','))):
            all_cuisines.append(i.split(',')[x].strip().upper())
            
print("\n\nNumber of Unique Cuisines : ", len(pd.Series(all_cuisines).unique()))
print("\n\nUnique Titles:\n", pd.Series(all_cuisines).unique())

all_cuisines = list(pd.Series(all_cuisines).unique())



Maximum cuisines in a cell :  8


Number of Unique Cuisines :  130


Unique Titles:
 ['MALWANI' 'GOAN' 'NORTH INDIAN' 'ASIAN' 'MODERN INDIAN' 'JAPANESE'
 'CHINESE' 'BIRYANI' 'HYDERABADI' 'TIBETAN' 'DESSERTS' 'SEAFOOD' 'CAFE'
 'PIZZA' 'BURGER' 'BAR FOOD' 'SOUTH INDIAN' 'FAST FOOD' 'BEVERAGES'
 'ARABIAN' 'MUGHLAI' 'MAHARASHTRIAN' 'PARSI' 'THAI' 'BAKERY' 'MOMOS'
 'CONTINENTAL' 'EUROPEAN' 'ROLLS' 'ANDHRA' 'ITALIAN' 'BBQ' 'FINGER FOOD'
 'TEA' 'AMERICAN' 'HEALTHY FOOD' 'COFFEE' 'INDONESIAN' 'KOREAN' 'NEPALESE'
 'ICE CREAM' 'MEXICAN' 'KERALA' 'INDIAN' 'MITHAI' 'STREET FOOD'
 'MALAYSIAN' 'VIETNAMESE' 'IRANIAN' 'KEBAB' 'JUICES' 'SANDWICH'
 'MEDITERRANEAN' 'SALAD' 'GUJARATI' 'RAJASTHANI' 'TEX-MEX' 'ROAST CHICKEN'
 'BURMESE' 'CHETTINAD' 'NORTH EASTERN' 'LEBANESE' 'COFFEE AND TEA' 'GRILL'
 '' 'BIHARI' 'BENGALI' 'LUCKNOWI' 'AWADHI' 'STEAK' 'FRENCH' 'PORTUGUESE'
 'WRAPS' 'SRI LANKAN' 'ORIYA' 'ETHIOPIAN' 'KONKAN' 'SUSHI' 'SPANISH'
 'RUSSIAN' 'MANGALOREAN' 'TURKISH' 'BUBBLE TEA' 'AFGHAN' 'NAGA'
 'SI

In [76]:
# Analysing CITY

all_cities = list(data_temp['CITY'])

for i in range(len(all_cities)):
    if type(all_cities[i]) == float:
        all_cities[i] = 'NOT AVAILABLE'
    all_cities[i] = all_cities[i].strip().upper()
    
print("\n\nNumber of Unique cities (Including NOT AVAILABLE): ", len(pd.Series(all_cities).unique()))
print("\n\nUnique Cities:\n", pd.Series(all_cities).unique())

all_cities = list(pd.Series(all_cities).unique())



Number of Unique cities (Including NOT AVAILABLE):  445


Unique Cities:
 ['THANE' 'CHENNAI' 'MUMBAI' 'BANGALORE' 'GURGAON' 'HYDERABAD' 'KOCHI'
 'THANE WEST' 'ANDHERI LOKHANDWALA' 'NEW DELHI' 'ANDHERI WEST'
 'MALAD EAST' '682036' 'BANGALOR' 'NAVI MUMBAI' 'BANDRA WEST' 'DELHI'
 'NOIDA' 'BANGALORE-560066' 'SECUNDERABAD' 'NOT AVAILABLE' 'INDIA'
 'MADHURANAGAR' 'CHENNAI TEYNAMPET' 'FARIDABAD' 'CHEMBUR.' 'MAHARASHTRA'
 'OPP GURUDWARA SHAKURPUR' 'TELAGANA LAND LINE:040-48507016' 'GHAZIABAD'
 'KARNATAKA' 'KERALA' 'EDAPPALLY' 'KADAVANTHRA' 'ERNAKULAM CIRCLE KOCHI'
 'BENGALORE' 'NEAR RELIANCE FRESH' 'KILPAUK' 'BENGALURU' 'KOTHAGUDA'
 'GOREGAON WEST' 'BANGLORE' 'TAMIL NADU' 'KAKKANAD' 'KOCHI ELAMKULAM'
 'OUTER RING ROAD' 'MULUND EAST'
 'SECUNDERABAD MAIN ROAD NEAR SIGNAL NMREC COLLEGE' 'TELANGANA'
 'PONNURUNI KOCHI' 'GACHIBOWLI' 'SEMMANCHERI'
 '5TH MAIN TEACHERS COLONY KORAMANGALA BLOCK 1 BANGALORE 560034'
 'MUMBAI MAHIM' 'POWAI (NEXT TO POWAI PLAZA)' 'DOMBIVALI EAST'
 'KOCHI VYTTILA' 'KANDIVA

In [77]:
# Cleaning LOCALITY

all_localities = list(data_temp['LOCALITY'])

for i in range(len(all_localities)):
    if type(all_localities[i]) == float:
        all_localities[i] = 'NOT AVAILABLE'
    all_localities[i] = all_localities[i].strip().upper()
    
print("\n\nNumber of Unique Localities (Including NOT AVAILABLE): ", len(pd.Series(all_localities).unique()))
print("\n\nUnique Localities:\n", pd.Series(all_localities).unique())

all_localities = list(pd.Series(all_localities).unique())    



Number of Unique Localities (Including NOT AVAILABLE):  1611


Unique Localities:
 ['DOMBIVALI EAST' 'RAMAPURAM' 'SALIGRAMAM' ... 'OFF CARTER ROAD'
 'SRM BACK GATE' 'PERRY CROSS ROAD']


# Data Cleaning

Now will proceed to cleaning the training and test sets.

The following steps are performed for bot training_set and test_set

*Splitting TITLE and CUISINES in to new feature sets and replacing NANs/empty cells with text "NONE" .

*Replacing NANs in CITY and LOCALITY with text "NOT AVAILABLE"

*Converting RATING type to float

*Cleaning the VOTES column and converting it to integers.

In [78]:
# Cleaning Training Set

In [79]:
# TITLE

titles = list(df_train['TITLE'])

# Since Maximum number of titles in a cell is 2 we will split title in to 2 columns

T1 = []
T2 = []

for i in titles:
    T1.append(i.split(',')[0].strip().upper())
    try:
        T2.append(i.split(',')[1].strip().upper())
    except:
        T2.append('NONE')

# appending NONE to Unique titles list
all_titles.append('NONE')

In [80]:
#Cleaning CUISINES 

cuisines = list(df_train['CUISINES'])

# Since Maximum number of cuisines in a cell is 8 we will split title in to 8 columns

   
C1 = []
C2 = []
C3 = []
C4 = []
C5 = []
C6 = []
C7 = []
C8 = []

for i in cuisines:
    try:
        C1.append(i.split(',')[0].strip().upper())
    except:
        C1.append('NONE')
    try:
        C2.append(i.split(',')[0].strip().upper())
    except:
        C2.append('NONE')
    try:
        C3.append(i.split(',')[0].strip().upper())
    except:
        C3.append('NONE') 
    try:
        C4.append(i.split(',')[0].strip().upper())
    except:
        C4.append('NONE')
    try:
        C5.append(i.split(',')[0].strip().upper())
    except:
        C5.append('NONE')
    try:
        C6.append(i.split(',')[0].strip().upper())
    except:
        C6.append('NONE')
    try:
        C7.append(i.split(',')[0].strip().upper())
    except:
        C7.append('NONE')
    try:
        C8.append(i.split(',')[0].strip().upper())
    except:
        C8.append('NONE') 

# appending NONE to Unique cuisines list
all_cuisines.append('NONE')
        

In [81]:
# Cleaning CITY

cities = list(df_train['CITY'])

for i in range(len(cities)):
    if type(cities[i]) == float:
        cities[i] = 'NOT AVAILABLE'
    cities[i] = cities[i].strip().upper()    

In [82]:
# Cleaning LOCALITY

localities = list(df_train['LOCALITY'])

for i in range(len(localities)):
    if type(localities[i]) == float:
        localities[i] = 'NOT AVAILABLE'
    localities[i] = localities[i].strip().upper()    

In [83]:
# Cleaning Rating

rates = list(df_train['RATING'])

for i in range(len(rates)):
    try:
        rates[i] = float(rates[i])
    except:
        rates[i] = np.nan

In [84]:
# Votes

votes = list(df_train['VOTES'])

for i in range(len(votes)):
    try:
        votes[i] = int(votes[i].split(" ")[0].strip())
    except:
        pass

In [85]:
new_data_train = {}

new_data_train['TITLE1'] = T1
new_data_train['TITLE2'] = T2
new_data_train['RESTAURANT_ID'] = df_train["RESTAURANT_ID"]
new_data_train['CUISINE1'] = C1
new_data_train['CUISINE2'] = C2
new_data_train['CUISINE3'] = C3
new_data_train['CUISINE4'] = C4
new_data_train['CUISINE5'] = C5
new_data_train['CUISINE6'] = C6
new_data_train['CUISINE7'] = C7
new_data_train['CUISINE8'] = C8
new_data_train['CITY'] = cities
new_data_train['LOCALITY'] = localities
new_data_train['RATING'] = rates
new_data_train['VOTES'] = votes
new_data_train['COST'] = df_train["COST"]

new_data_train = pd.DataFrame(new_data_train)

In [86]:
# Cleaning Test Set

In [87]:
# TITLE

titles = list(df_test['TITLE'])

# Since Maximum number of titles in a cell is 2 will will split title in to 2 columns

T1 = []
T2 = []

for i in titles:
    T1.append(i.split(',')[0].strip().upper())
    try:
        T2.append(i.split(',')[1].strip().upper())
    except:
        T2.append('NONE')

In [88]:
#Cleaning CUISINES 

cuisines = list(df_test['CUISINES'])
   
# Since Maximum number of cuisines in a cell is 8 we will split title in to 8 columns
   
C1 = []
C2 = []
C3 = []
C4 = []
C5 = []
C6 = []
C7 = []
C8 = []


for i in cuisines:
        try :
            C1.append(i.split(',')[0].strip().upper())
        except :
            C1.append('NONE')
        try :
            C2.append(i.split(',')[1].strip().upper())
        except :
            C2.append('NONE')
        try :
            C3.append(i.split(',')[2].strip().upper())
        except :
            C3.append('NONE')
        try :
            C4.append(i.split(',')[3].strip().upper())
        except :
            C4.append('NONE')
        try :
            C5.append(i.split(',')[4].strip().upper())
        except :
            C5.append('NONE')
        try :
            C6.append(i.split(',')[5].strip().upper())
        except :
            C6.append('NONE')
        try :
            C7.append(i.split(',')[6].strip().upper())
        except :
            C7.append('NONE')
        try :
            C8.append(i.split(',')[7].strip().upper())
        except :
            C8.append('NONE')

In [89]:
# Cleaning CITY

cities = list(df_test['CITY'])

for i in range(len(cities)):
    if type(cities[i]) == float:
        cities[i] = 'NOT AVAILABLE'
    cities[i] = cities[i].strip().upper() 

In [90]:
# Cleaning LOCALITY

localities = list(df_test['LOCALITY'])

for i in range(len(localities)):
    if type(localities[i]) == float:
        localities[i] = 'NOT AVAILABLE'
    localities[i] = localities[i].strip().upper() 

In [91]:
#Cleaning Rating

rates = list(df_test['RATING'])

for i in range(len(rates)) :
    try:
        rates[i] = float(rates[i])
    except :
        rates[i] = np.nan

In [92]:
# Votes
       
votes = list(df_test['VOTES'])

for i in range(len(votes)) :
    try:
        votes[i] = int(votes[i].split(" ")[0].strip())
    except :
        pass  

In [93]:
new_data_test = {}

new_data_test['TITLE1'] = T1
new_data_test['TITLE2'] = T2
new_data_test['RESTAURANT_ID'] = df_test["RESTAURANT_ID"]
new_data_test['CUISINE1'] = C1
new_data_test['CUISINE2'] = C2
new_data_test['CUISINE3'] = C3
new_data_test['CUISINE4'] = C4
new_data_test['CUISINE5'] = C5
new_data_test['CUISINE6'] = C6
new_data_test['CUISINE7'] = C7
new_data_test['CUISINE8'] = C8
new_data_test['CITY'] = cities
new_data_test['LOCALITY'] = localities
new_data_test['RATING'] = rates
new_data_test['VOTES'] = votes

new_data_test = pd.DataFrame(new_data_test)

print("\n\nnew_data_train: \n", new_data_train.head())
print("\n\nnew_data_test: \n", new_data_test.head())



new_data_train: 
            TITLE1 TITLE2  RESTAURANT_ID      CUISINE1      CUISINE2  \
0   CASUAL DINING   NONE           9438       MALWANI       MALWANI   
1   CASUAL DINING    BAR          13198         ASIAN         ASIAN   
2   CASUAL DINING   NONE          10915  NORTH INDIAN  NORTH INDIAN   
3     QUICK BITES   NONE           6346       TIBETAN       TIBETAN   
4  DESSERT PARLOR   NONE          15387      DESSERTS      DESSERTS   

       CUISINE3      CUISINE4      CUISINE5      CUISINE6      CUISINE7  \
0       MALWANI       MALWANI       MALWANI       MALWANI       MALWANI   
1         ASIAN         ASIAN         ASIAN         ASIAN         ASIAN   
2  NORTH INDIAN  NORTH INDIAN  NORTH INDIAN  NORTH INDIAN  NORTH INDIAN   
3       TIBETAN       TIBETAN       TIBETAN       TIBETAN       TIBETAN   
4      DESSERTS      DESSERTS      DESSERTS      DESSERTS      DESSERTS   

       CUISINE8     CITY        LOCALITY  RATING  VOTES  COST  
0       MALWANI    THANE  DOMBIVALI EA

# Data Preprocessing

In [94]:
#Dealing with missing values

In [95]:
# Training set

print("\n\nMissing Values in Training Set\n","################################################")
print("\nContains Nan/Empty cells : ",new_data_train.isnull().values.any())
print("\nTotal empty cells by column\n","_"*60,"\n",new_data_train.isnull().sum())

new_data_train.fillna(0,inplace = True)

print("\n\nAfter Filling 0:\n","_"*60,"\n")
print("\nContains Nan/Empty cells : ", new_data_train.isnull().values.any())



Missing Values in Training Set
 ################################################

Contains Nan/Empty cells :  True

Total empty cells by column
 ____________________________________________________________ 
 TITLE1              0
TITLE2              0
RESTAURANT_ID       0
CUISINE1            0
CUISINE2            0
CUISINE3            0
CUISINE4            0
CUISINE5            0
CUISINE6            0
CUISINE7            0
CUISINE8            0
CITY                0
LOCALITY            0
RATING           1204
VOTES            1204
COST                0
dtype: int64


After Filling 0:
 ____________________________________________________________ 


Contains Nan/Empty cells :  False


In [96]:
# Test set

print("\n\nMissing Values in Test Set\n","################################################")
print("\nContains Nan/Empty cells : ",new_data_test.isnull().values.any())
print("\nTotal empty cells by column\n","_"*60,"\n",new_data_test.isnull().sum())

new_data_test.fillna(0,inplace = True)

print("\n\nAfter Filling 0:\n","_"*60,"\n")
print("\nContains Nan/Empty cells : ", new_data_test.isnull().values.any())



Missing Values in Test Set
 ################################################

Contains Nan/Empty cells :  True

Total empty cells by column
 ____________________________________________________________ 
 TITLE1             0
TITLE2             0
RESTAURANT_ID      0
CUISINE1           0
CUISINE2           0
CUISINE3           0
CUISINE4           0
CUISINE5           0
CUISINE6           0
CUISINE7           0
CUISINE8           0
CITY               0
LOCALITY           0
RATING           402
VOTES            402
dtype: int64


After Filling 0:
 ____________________________________________________________ 


Contains Nan/Empty cells :  False


In [97]:
# Encoding Categorical Variables

from sklearn.preprocessing import LabelEncoder

le_titles = LabelEncoder()
le_cuisines = LabelEncoder()

le_city = LabelEncoder()
le_locality = LabelEncoder()

le_titles.fit(all_titles)
le_cuisines.fit(all_cuisines)

le_city.fit(all_cities)
le_locality.fit(all_localities)



# Train Set  

new_data_train['TITLE1'] = le_titles.transform(new_data_train['TITLE1'])
new_data_train['TITLE2'] = le_titles.transform(new_data_train['TITLE2'])


new_data_train['CUISINE1'] = le_cuisines.transform(new_data_train['CUISINE1'])
new_data_train['CUISINE2'] = le_cuisines.transform(new_data_train['CUISINE2'])
new_data_train['CUISINE3'] = le_cuisines.transform(new_data_train['CUISINE3'])
new_data_train['CUISINE4'] = le_cuisines.transform(new_data_train['CUISINE4'])
new_data_train['CUISINE5'] = le_cuisines.transform(new_data_train['CUISINE5'])
new_data_train['CUISINE6'] = le_cuisines.transform(new_data_train['CUISINE6'])
new_data_train['CUISINE7'] = le_cuisines.transform(new_data_train['CUISINE7'])
new_data_train['CUISINE8'] = le_cuisines.transform(new_data_train['CUISINE8'])


new_data_train['CITY'] = le_city.transform(new_data_train['CITY'])
new_data_train['LOCALITY'] = le_locality.transform(new_data_train['LOCALITY'])

In [98]:
# Test Set

new_data_test['TITLE1'] = le_titles.transform(new_data_test['TITLE1'])
new_data_test['TITLE2'] = le_titles.transform(new_data_test['TITLE2'])


new_data_test['CUISINE1'] = le_cuisines.transform(new_data_test['CUISINE1'])
new_data_test['CUISINE2'] = le_cuisines.transform(new_data_test['CUISINE2'])
new_data_test['CUISINE3'] = le_cuisines.transform(new_data_test['CUISINE3'])
new_data_test['CUISINE4'] = le_cuisines.transform(new_data_test['CUISINE4'])
new_data_test['CUISINE5'] = le_cuisines.transform(new_data_test['CUISINE5'])
new_data_test['CUISINE6'] = le_cuisines.transform(new_data_test['CUISINE6'])
new_data_test['CUISINE7'] = le_cuisines.transform(new_data_test['CUISINE7'])
new_data_test['CUISINE8'] = le_cuisines.transform(new_data_test['CUISINE8'])


new_data_test['CITY'] = le_city.transform(new_data_test['CITY'])
new_data_test['LOCALITY'] = le_locality.transform(new_data_test['LOCALITY'])


In [99]:
# Classifying Independent and Dependent Features

# Dependent Variable
y_train = new_data_train.iloc[:,-1].values

# Independent Variables
x_train = new_data_train.iloc[:,0:-1].values

# Independent Variables for Test Set
x_test = new_data_test.iloc[:,:].values

In [100]:
sc = StandardScaler()

x_train = sc.fit_transform(x_train)

x_test = sc.fit_transform(x_test)

y_train = y_train.reshape((len(y_train),1))

y_train = sc.fit_transform(y_train)

y_train = y_train.ravel()

In [101]:
lr = LinearRegression()

lr.fit(x_train,y_train)

y_pred=lr.predict(x_test)

lrscore=lr.score(x_train,y_train)
print(lrscore*100)

19.523610058561992


In [102]:
## Using GridSearchCV to find out the best parameters of the KNeighborsRegressor model
neighbors = {"n_neighbors":range(1,30)}
knr=KNeighborsRegressor()
gknr = GridSearchCV(knr,neighbors,cv=10)
gknr.fit(x_train,y_train)
gknr.best_params_

{'n_neighbors': 8}

In [103]:
knr=KNeighborsRegressor(n_neighbors=8)
knr.fit(x_train,y_train)
knrscore= knr.score(x_train,y_train)
print(knrscore*100)

51.5535937078876


In [104]:
rd = Ridge()

rd.fit(x_train,y_train)

y_pred=rd.predict(x_test)

rdscore=rd.score(x_train,y_train)
print(rdscore*100)

19.523609996279767


In [105]:
knr = KNeighborsRegressor()

knr.fit(x_train,y_train)

y_pred=knr.predict(x_test)

knrscore=knr.score(x_train,y_train)
print(knrscore*100)

57.517714394941976


In [106]:
svr = SVR()

svr.fit(x_train,y_train)

y_pred=svr.predict(x_test)

svrscore=svr.score(x_train,y_train)
print(svrscore*100)

28.963913546707467


In [107]:
gbr = GradientBoostingRegressor()

gbr.fit(x_train,y_train)

y_pred=gbr.predict(x_test)

gbrscore=gbr.score(x_train,y_train)
print(gbrscore*100)

71.58155119231205


In [108]:
# to check whether our result gets boosted or not
from sklearn.ensemble import RandomForestRegressor

In [109]:
rf =RandomForestRegressor()
rf.fit(x_train,y_train)
rfscore=rf.score(x_train,y_train)
print(rfscore*100)

95.92812119757073


In [110]:
from sklearn.externals import joblib

In [111]:
#Save the model as a pickle in a file
joblib.dump(rf,"rfresfood.pkl")

['rfresfood.pkl']

In [112]:
model=joblib.load("rfresfood.pkl")